# Time evolution with a time-dependent Hamiltonian

In [1]:
using DifferentialEquations
using ITensors
using ITensorNetworks
using KrylovKit: exponentiate
using LinearAlgebra
using Test

In this notebook we will demonstrate simulating time evolution for a spin model with a time dependent Hamiltonian. on a lattice with a tree-like geometry using the [TDVP](https://arxiv.org/abs/1408.5056) algorithm.

As an example, we will consider a spin-1/2 system with time-dependent next-to-nearest neighbor Heisenberg interactions

$$
H(t) = J_1(t) \sum_{\langle i, j \rangle} \vec{S}_i \cdot \vec{S}_j + J_2(t) \sum_{\langle\langle i, j \rangle\rangle} \vec{S}_i \cdot \vec{S}_j + h(t) \sum_i S^z_i,
$$

on a spin system defined on a small comb tree with 6 sites

<center><img align="center" width="200" src="./fig/small_comb_tree.svg"></center>

### Representing a time-dependent Hamiltonian

Consider a simple time-dependent Hamiltonian, which can be written as a sum of static time-independent operators, each of which is mutliplied by its own time-dependent prefactor:

$$H(t) = \sum_i f_i(t) H_i$$

To represent such an operator in a tree tensor network form, we can convert the static operators $H_i$ to `TTN` objects in the usual way, and just keep track of a list of the time-dependent coefficients and the static `TTN` operators.

As a simple example, we consider a sum of two terms that each have a harmonic time-dependence with a different frequency:

In [2]:
# define system geometry
c = named_comb_tree((3, 2))
root_vertex = (3, 2)
s = siteinds("S=1/2", c)

# initialize time-dependent prefactors
ωs = [0.1, 0.2]
fs = [t -> cos(ω * t) for ω in ωs]

# initialize static Hamiltonian terms
J₁ = 1.0
J₂ = 0.1
Hs = [heisenberg(c; J1=1.0, J2=0.0), heisenberg(c; J1=0.0, J2=0.1)]
Hs = [TTN(H, s) for H in Hs];

### Solvers for time-dependent problems

Explaining how to define a local solver for time-dependent problems first requires some discussion of the internals of the sweeping routines in ITensorNetworks.jl. Whenever a local update is executed in a sweeping algorithm, an effective Hamiltonian is constructed which embodies the action of the system Hamiltonian on the local region to be updated while taking into account the rest of the network in the form of environments. The action of this effective Hamiltonian, say `H`, on the region to be updated, represented by a single tensor `ψ`, is executed by evaluating `H` on `ψ`: `H(ψ)`.

A time dependent operator in ITensorNetworks.jl is represented as an instance of a `TimeDependentSum`, which essentially contains a list of static effective operators and a list of their time-dependent prefactors. Given a `TimeDependentSum`, `H`, the effective Hamiltonian operator at a given time `t` can be extracted by evaluating `H` as `H_t = H(t)`, after which `H_t(ψ)` represents the action of the effective Hamiltonian at time `t` on a local region `ψ`.

The first step in defining time evolution solvers for time-dependent problems is to define a solver which takes a `TimeDependentSum` encoding all time dependence as an input, and returns the effect of the exponential of this operator evaluated at the current time (always passed to a local solver as a `current_time` keyword argument) on a given tensor. As an example we give two such solvers, using a DifferentialEquations.jl integrator and a KrylovKit.jl exponentiator respectively:

In [3]:
function ode_solver(
    H::TimeDependentSum,
    time_step,
    ψ0;
    current_time=0.0,
    outputlevel=0,
    solver_alg=Tsit5(),
    kwargs...,
)
    if outputlevel ≥ 3
        println("    In ODE solver, current_time = $current_time, time_step = $time_step")
    end

    time_span = (current_time, current_time + time_step)
    u0, ITensor_from_vec = to_vec(ψ0)
    f(ψ::ITensor, p, t) = H(t)(ψ)
    f(u::Vector, p, t) = to_vec(f(ITensor_from_vec(u), p, t))[1]
    prob = ODEProblem(f, u0, time_span)
    sol = solve(prob, solver_alg; kwargs...)
    ut = sol.u[end]
    return ITensor_from_vec(ut), nothing
end

function krylov_solver(
    H::TimeDependentSum,
    time_step,
    ψ0;
    current_time=0.0,
    outputlevel=0,
    kwargs...,
)
    if outputlevel ≥ 3
        println("    In Krylov solver, current_time = $current_time, time_step = $time_step")
    end
    ψt, info = exponentiate(H(current_time), time_step, ψ0; kwargs...)
    return ψt, info
end;

The next step in defining a TDVP solver for a time-dependent problem is to wrap these solvers in corresponding methods which are compatible with the internal standard solver interface. This interface (assuming ITensorNetworks.jl@0.3.9) provides the solver with a (list of) static effective Hamiltonian(s) representing (a sum of) time-independent Hamiltonian(s). We can add the appropriate time dependence by wrapping these static effective operators in a `TimeDependentSum` using the time-dependence specified above.

In [4]:
ode_alg = Tsit5()
ode_kwargs = (; reltol=1e-8, abstol=1e-8)
function tdvp_ode_solver(Hs, ψ0; time_step, kwargs...)
    psi_t, info = ode_solver(
        -im * TimeDependentSum(fs, Hs), time_step, ψ0; solver_alg=ode_alg, ode_kwargs...
    )
    return psi_t, (; info)
end

krylov_kwargs = (; tol=1e-8, eager=true)
function tdvp_krylov_solver(Hs, ψ0; time_step, ishermitian=false, issymmetric=false, kwargs...)
    psi_t, info = krylov_solver(
        -im * TimeDependentSum(fs, Hs),
        time_step,
        ψ0;
        krylov_kwargs...,
        ishermitian,
        issymmetric,
    )
    return psi_t, (; info)
end;

### Integrating the time-dependent problem

Having defined our solvers, we can now perform the time integration using the TDVP sweeping algorithm and compare the result to an exact time integration for the full dense quantum state.

In [5]:
time_step = 0.1
time_total = 1.0

nsite = 2
maxdim = 100
cutoff = 1e-8

# start from a product state
ψ0 = TTN(ComplexF64, s, v -> iseven(sum(isodd.(v))) ? "↑" : "↓")

ψt_ode = tdvp(tdvp_ode_solver, Hs, time_total, ψ0; time_step, maxdim, cutoff, nsite)

ψt_krylov = tdvp(tdvp_krylov_solver, Hs, time_total, ψ0; time_step, cutoff, nsite)

ψt_full, _ = tdvp_ode_solver(contract.(Hs), contract(ψ0); time_step=time_total)

@show norm(ψ0) ≈ 1
@show norm(ψt_ode) ≈ 1
@show norm(ψt_krylov) ≈ 1
@show norm(ψt_full) ≈ 1

ode_err = norm(contract(ψt_ode) - ψt_full)
krylov_err = norm(contract(ψt_krylov) - ψt_full)

@show krylov_err > ode_err
@show ode_err < 1e-2
@show krylov_err < 1e-2;

norm(ψ0) ≈ 1 = true
norm(ψt_ode) ≈ 1 = true
norm(ψt_krylov) ≈ 1 = true
norm(ψt_full) ≈ 1 = true
krylov_err > ode_err = true
ode_err < 0.01 = true
krylov_err < 0.01 = true
